In [56]:
# Importing packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from functools import reduce
from collections import defaultdict
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_decomposition import PLSSVD
from sklearn.compose import ColumnTransformer

# displaying all the rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# setting random seed:
RANDOM_SEED = 42

## 1. EDA

In [57]:
# Load the data:
data = pd.read_csv('/Users/bangxixiao/Desktop/python_projects/DATA1030_MIDTERM_PROJECT/data/application_data.csv', 
                   encoding='UTF-8')

In [3]:
print('The credit default data has {} rows and {} columns.'.format(data.shape[0], data.shape[1]))

The credit default data has 307511 rows and 122 columns.


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


The data has 307511 entries and 122 columns, where 65 columns are identified as float, 41 columns as integer and 16 columns as object (string).

In [5]:
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

We noticed that there exists a small proportion of missing data.

In [6]:
# describe the data:
data.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,104582.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307509.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,134133.000000,3.068510e+05,246546.000000,151450.00000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,159080.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.00000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,12.061091,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,2.152665,2.052463,2.031521,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,0.502130,5.143927e-01,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,0.066333,0.100775,0.107399,0.008809,0.028358,0.114231,0.087543,0.977065,0.759637,0.042553,0.074490,0.145193,0.222315,0.228058,0.064958,0.105645,0.105975,0.008076,0.027022,0.117850,0.087955,0.97

The columns could be separated into different categories by looking at the prefix of column names:

AMT: data related to specific numbers such as loan annuity, annuity of previous application and so on.

APARTMENTS: apartment-related information

BASEMENTAREA: basement information about building where the clients live

CHANEL: Through which channel we acquired the client on the previous application

CNT: count data such the number of children the client has

CODE: includes gender and why previous application rejected

COMMONAREA: common area information about building where the clients live

DAYS: includes date-related information such as birth, employment

DEF: clients' social surroundings count

ELEVATORS: living condition of clients

EMERGENCYSTATE: living condition of clients

ENTRANCES: living condition of clients

EXT: normalized score from external data source

FLAG: consists of bool values

FLOORSMAX/FLOORSMIN: living condition of clients

HOUR: data regarding the time, for example, the exact hour the client apply for the loan

LANDAREA: living condition of the clients

LIVE: if the client's contact address match with the work address

LIVINGAPARTMENTS/LIVINGAREA: living condition of clients

NAME: mostly strings data regarding the client's personal information

NFLAG: bool-like data

NONLIVINGAPARTMENT/NONLIVINGAREA: living condition of the clients

OBS: number of observations of clients' social surroundings

OCCUPATION: kind of occupation client has

ORGANIZATION: type of organization where client works

OWN: age of client's car

PRODUCT: detailed product combination of the previous application

RATE: fractional number

REG: bool type data, regarding the match of client's permanent address

SELLERPLACE: selling area of seller place of the previous application

SK_: ID-like data

TARGET: target variable

TOTALAREA: living condition of client

WALLSMATERIAL: living condition of client

WEEKDAY: which day in a week the client applies for a loan

YEARS: living condition of client

In [61]:
(data.isnull().sum(axis=0) / data.shape[0]).sort_values()

SK_ID_CURR                      0.000000
HOUR_APPR_PROCESS_START         0.000000
REG_REGION_NOT_WORK_REGION      0.000000
LIVE_REGION_NOT_WORK_REGION     0.000000
REG_CITY_NOT_LIVE_CITY          0.000000
REG_CITY_NOT_WORK_CITY          0.000000
LIVE_CITY_NOT_WORK_CITY         0.000000
ORGANIZATION_TYPE               0.000000
FLAG_DOCUMENT_21                0.000000
FLAG_DOCUMENT_20                0.000000
FLAG_DOCUMENT_19                0.000000
FLAG_DOCUMENT_18                0.000000
FLAG_DOCUMENT_17                0.000000
FLAG_DOCUMENT_16                0.000000
FLAG_DOCUMENT_15                0.000000
FLAG_DOCUMENT_14                0.000000
FLAG_DOCUMENT_13                0.000000
FLAG_DOCUMENT_12                0.000000
FLAG_DOCUMENT_11                0.000000
FLAG_DOCUMENT_10                0.000000
FLAG_DOCUMENT_9                 0.000000
FLAG_DOCUMENT_8                 0.000000
FLAG_DOCUMENT_7                 0.000000
FLAG_DOCUMENT_6                 0.000000
FLAG_DOCUMENT_5 

In [62]:
drop_columns = []
drop_columns += list(data.columns[data.isnull().sum(axis=0) / data.shape[0] > 0.35])

In [63]:
data['EXT_SOURCE_3'].fillna(data['EXT_SOURCE_3'].median(), inplace=True)

In [64]:
cred_bureau_cols = [
    col 
    for col in data.columns 
    if col.startswith("AMT_REQ_CREDIT_BUREAU_")
]
for col in cred_bureau_cols:
    data[col] = data[col].fillna(0.0)
drop_columns.extend(['AMT_REQ_CREDIT_BUREAU_YEAR', 'DAYS_LAST_PHONE_CHANGE'])
drop_columns.extend(['NAME_TYPE_SUITE', 'OCCUPATION_TYPE'])

In [81]:
data['NAME_INCOME_TYPE'].value_counts()

Working        250482
Retired         55016
Unemployed         19
Student            18
Businessman        10
Name: NAME_INCOME_TYPE, dtype: int64

In [65]:
replace_dict = {
    "Commercial associate" : "Working", 
    "State servant"        : "Working", 
    "Maternity leave"      : "Working", 
    "Pensioner"            : "Retired",
    'Businessman': 'Working',
}
data['NAME_INCOME_TYPE'] = data['NAME_INCOME_TYPE'].replace(replace_dict)


replace_dict = {
    "Secondary / secondary special" : "Secondary_Education",
    "Higher education"              : "Higher_Education",
    "Incomplete higher"             : "Secondary_Education",
    "Lower secondary"               : "Lower_Secondary",
    "Academic degree"               : "Higher Education"
}
data['NAME_EDUCATION_TYPE'] = data['NAME_EDUCATION_TYPE'].replace(replace_dict)


replace_dict = {
    "Single / not married" : "Single",
    "Civil marriage"       : "Married",
    "Separated"            : "Married",
    "Widow"                : "Married",
    "Unknown"              : "Married"
}
data['NAME_FAMILY_STATUS'].replace(replace_dict, inplace=True)


replace_dict = {
    "House / apartment"   : "Owned",
    "With parents"        : "Owned",
    "Municipal apartment" : "Freehold",
    "Office apartment"    : "Freehold",
    "Rented apartment"    : "Rented",
    "Co-op"               : "Rented"
}
data['NAME_HOUSING_TYPE'].replace(replace_dict, inplace=True)

In [66]:
flag_doc_cols = ['FLAG_DOCUMENT_{}'.format(x) for x in range(2, 22)]
data['CNT_DOCUMENT'] = data[flag_doc_cols].sum(axis=1)
data.drop(columns=flag_doc_cols, inplace=True)

In [67]:
data['CODE_GENDER'].replace({"M": 1, "F": 0, "XNA": 1}, inplace=True)
data['CODE_GENDER'] = data['CODE_GENDER'].astype('int64')
data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(lambda x: int(x * (-1) / 365))
data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].apply(lambda x: int(x * (-1) / 365))

In [68]:
data.drop(columns=[
    'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'SK_ID_CURR'
], inplace=True)
data.drop(columns=drop_columns, inplace=True)

In [69]:
data.dtypes.sort_values()

TARGET                           int64
LIVE_CITY_NOT_WORK_CITY          int64
REG_CITY_NOT_WORK_CITY           int64
REG_CITY_NOT_LIVE_CITY           int64
LIVE_REGION_NOT_WORK_REGION      int64
REG_REGION_NOT_WORK_REGION       int64
REG_REGION_NOT_LIVE_REGION       int64
REGION_RATING_CLIENT_W_CITY      int64
REGION_RATING_CLIENT             int64
FLAG_EMAIL                       int64
FLAG_CONT_MOBILE                 int64
FLAG_WORK_PHONE                  int64
FLAG_EMP_PHONE                   int64
FLAG_MOBIL                       int64
DAYS_ID_PUBLISH                  int64
DAYS_EMPLOYED                    int64
FLAG_PHONE                       int64
CNT_DOCUMENT                     int64
CODE_GENDER                      int64
CNT_CHILDREN                     int64
DAYS_BIRTH                       int64
AMT_REQ_CREDIT_BUREAU_MON      float64
AMT_REQ_CREDIT_BUREAU_WEEK     float64
AMT_REQ_CREDIT_BUREAU_DAY      float64
AMT_REQ_CREDIT_BUREAU_HOUR     float64
DEF_60_CNT_SOCIAL_CIRCLE 

In [79]:
ORDINAL_FEATURES = ['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']
CONTINUOUS_FEATURES = ['EXT_SOURCE_2',
                         'EXT_SOURCE_3',
                         'AMT_INCOME_TOTAL',
                         'AMT_CREDIT',
                         'AMT_ANNUITY',
                         'AMT_GOODS_PRICE',
                         'AMT_REQ_CREDIT_BUREAU_MON',
                         'AMT_REQ_CREDIT_BUREAU_DAY',
                         'AMT_REQ_CREDIT_BUREAU_WEEK',
                         'AMT_REQ_CREDIT_BUREAU_QRT',
                         'AMT_REQ_CREDIT_BUREAU_HOUR',
                         'OBS_30_CNT_SOCIAL_CIRCLE',
                         'DEF_30_CNT_SOCIAL_CIRCLE',
                         'OBS_60_CNT_SOCIAL_CIRCLE',
                         'DEF_60_CNT_SOCIAL_CIRCLE',
                         'CNT_CHILDREN',
                         'CNT_FAM_MEMBERS',
                         'CNT_DOCUMENT',
                         'DAYS_BIRTH',
                         'DAYS_EMPLOYED',
                         'DAYS_REGISTRATION',
                         'DAYS_ID_PUBLISH',
                         'REGION_POPULATION_RELATIVE']
CATEGORICAL_FEATURES = ['NAME_CONTRACT_TYPE',
                         'NAME_INCOME_TYPE',
                         'NAME_EDUCATION_TYPE',
                         'NAME_FAMILY_STATUS',
                         'NAME_HOUSING_TYPE',
                         'FLAG_OWN_CAR',
                         'FLAG_OWN_REALTY',
                         'FLAG_EMP_PHONE',
                         'FLAG_WORK_PHONE',
                         'FLAG_PHONE',
                         'FLAG_EMAIL',
                         'CODE_GENDER',
                         'FLAG_MOBIL',
                         'FLAG_CONT_MOBILE',
                         'ORGANIZATION_TYPE',
                         'LIVE_REGION_NOT_WORK_REGION',
                         'LIVE_CITY_NOT_WORK_CITY',
                         'REG_REGION_NOT_LIVE_REGION',
                         'REG_REGION_NOT_WORK_REGION',
                         'REG_CITY_NOT_LIVE_CITY',
                         'REG_CITY_NOT_WORK_CITY']

In [80]:
data = data.dropna()
data[CONTINUOUS_FEATURES + CATEGORICAL_FEATURES + ORDINAL_FEATURES + ['TARGET']].to_csv(
    '/Users/bangxixiao/Desktop/python_projects/DATA1030_MIDTERM_PROJECT/data/data_imputed_2.csv', 
    index=False
)